# PROJET IA for HumanForYou - Entraînement et Évaluation du Modèle

|Auteur|
|---|
|G. DUBOYS DE LAVIGERIE|
|T. VILLETTE|
|O. BOUSSARD|
|A. BRICON|

## Objectifs du Notebook

Ce notebook a pour objectif de développer plusieurs modèles d'IA en utilisant les données prétraitées et de les évaluer sur un ensemble de test. Les étapes comprennent :

1. **Division des données :** Séparation des données en ensembles d'entraînement et de test.
2. **Entraînement des modèles :** Développement de plusieurs modèles d'IA avec les données d'entraînement.
3. **Évaluation des modèles :** Mesure des performances de chaque modèle sur l'ensemble de test.
4. **Sélection du meilleur modèle :** Analyse des résultats pour choisir le modèle offrant les performances les plus prometteuses.

## Attendus

À la fin de ce notebook, le modèle sera prêt à être déployé, accompagné d'une évaluation détaillée de ses performances.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir exécuté le notebook `data_preprocessing.ipynb` pour garantir que les données sont prêtes pour l'entraînement du modèle.

### Préparation de l'environnement

In [ ]:
# Compatibilité entre Python 2 et Python 3
from __future__ import division, print_function, unicode_literals

# Imports nécessaires
import numpy as np
import os

# Assurer la stabilité du notebook entre différentes exécutions
np.random.seed(42)

# Affichage de figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Ajustements des paramètres de l'affichage des figures
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Définition du chemin où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "workflowDS"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)  # Assurez-vous que le dossier existe

# Fonction pour sauvegarder les figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Sauvegarde de la figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignorer les avertissements inutiles (voir SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### Importation des données traitées réalisée dans le notebook `data_preprocessing.ipynb`

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) # Retirer la limite de colonnes à afficher

def load_preprocessed_data(file_path='../datasets/data_preprocessing.csv'):
    return pd.read_csv(file_path)

# Charger les données prétraitées
result = load_preprocessed_data()
result.head()

### Division des données (jeu d'entraînement, jeu de test)
Cette étape consiste à séparer les données en un jeu d'entraînement (80%) et un jeu de test (20%) afin d'évaluer la capacité du modèle à généraliser sur de nouvelles données. 

- Le **jeu d'entraînement** est utilisé pour former le modèle, lui fournissant une quantité suffisante de données pour apprendre les motifs et les relations présents dans nos données.

- Le **jeu de test** permet de mesurer la performance du modèle sur des données non vues pendant l'entraînement, offrant ainsi une évaluation impartiale de sa capacité de généralisation.

Le choix de la division 80/20 n'est pas arbitraire. Il s'agit d'une règle empirique bien établie dans le domaine de l'apprentissage machine, offrant un équilibre optimal entre la taille de l'ensemble d'entraînement nécessaire pour un apprentissage efficace et la capacité à évaluer le modèle de manière significative sur le jeu de test.


In [ ]:
from sklearn.model_selection import train_test_split

# Supposons que nous voulons prédire 'cat_nom__Attrition_Yes'
y = result['cat_ord__Attrition']
X = result.drop(['cat_ord__Attrition'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Affichage d'un graphique
fig, ax = plt.subplots(figsize=(2, 5))

rect_train = plt.Rectangle((0, 0), 1, 0.8, color='red', alpha=0.5, label='Entraînement')
rect_test = plt.Rectangle((0, 0.8), 1, 0.2, color='blue', alpha=0.5, label='Test')

ax.add_patch(rect_train)
ax.add_patch(rect_test)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Répartition des Données")

ax.text(0.5, 0.4, f'{len(X_train)}', ha='center', va='center', color='black', fontsize=8)
ax.text(0.5, 0.9, f'{len(X_test)}', ha='center', va='center', color='black', fontsize=8)

ax.legend(loc='center')

plt.show()


### Choisir et Entraîner un modèle

Avant de plonger dans les détails, il est essentiel de comprendre comment choisir, entraîner et évaluer un modèle. Une étape cruciale consiste à analyser les performances du modèle sur un ensemble de test et à utiliser des techniques comme la validation croisée pour garantir une évaluation robuste du modèle.

**Fonction de validation croisée**

La fonction `train_evaluate_model` ci-dessous illustre un processus générique d'entraînement et d'évaluation de modèles de régression. Cela garantit une approche standardisée dans notre notebook, simplifiant ainsi le processus et assurant une cohérence d'évaluation, indépendamment de l'algorithme sous-jacent.

En utilisant la fonction `train_evaluate_model`, nous appliquons la validation croisée avec deux métriques :
- **RMSE (Root Mean Square Error)** : Mesure la moyenne des erreurs au carré entre prédictions et valeurs réelles, avec prise de la racine carrée de cette moyenne.
- **MAE (Mean Absolute Error)** : Autre métrique mesurant la moyenne des valeurs absolues des erreurs entre prédictions et valeurs réelles.

Cette approche renforce l'évaluation des modèles en utilisant `scikit-learn` pour calculer le RMSE, la MAE, et la validation croisée, fournissant une évaluation complète de la performance du modèle.

Il convient de noter que nous utilisons cette fonction pour chacun de nos modèles, assurant ainsi une comparaison équitable des performances entre les différentes approches algorithmiques que nous explorons.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def train_model(trainer):
    # Entraînement du modèle
    trainer.fit(X_train, y_train)
    
    some_data = X_train.iloc[:5]
    some_labels = y_train.iloc[:5]

    # Et on effectue la prédiction :
    print("Predictions:", trainer.predict(some_data))
    print("Labels:", list(some_labels)) # vraies valeurs

    # Prédictions sur les données de test
    trainer_predictions = trainer.predict(X_test)
    # Évaluation des prédictions sur les données de test
    trainer_mse = mean_squared_error(y_test, trainer_predictions)
    trainer_rmse = np.sqrt(trainer_mse)

    # Affichage des métriques d'évaluation
    print("MSE:", trainer_mse)
    print("RMSE:", trainer_rmse)

    # Validation croisée pour évaluer la performance du modèle
    scores = cross_val_score(trainer, X_test, y_test,
                             scoring="neg_mean_squared_error", cv=10)
    trainer_rmse_scores = np.sqrt(-scores)

    # Affichage des scores de validation croisée
    print("Cross-Validation Scores:", trainer_rmse_scores)

    # Affichage de la performance moyenne et de la variabilité du modèle
    print("Mean RMSE:", trainer_rmse_scores.mean())
    print("RMSE Standard Deviation:", trainer_rmse_scores.std())

#### Modèle de régression

##### Modèle de régression linéaire
Entraînons un modèle de [_régression linéaire_](https://fr.wikipedia.org/wiki/R%C3%A9gression_lin%C3%A9aire) avec nos données.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
train_model(lin_reg)

##### Modèle de régression par arbre de décision (DecisionTreeRegressor) 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create a DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)
train_model(tree_reg)


##### Modèle RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(max_depth=15, max_features='log2', n_estimators=26,
                      random_state=42)
train_model(forest_reg)

#### Modèle de classification

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, precision_recall_curve

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b-", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])


def display_scores(y_train_yes, y_train_pred, y_scores):
    conf_matrix =  confusion_matrix(y_train_yes, y_train_pred)    
    precision = precision_score(y_train_yes, y_train_pred)
    recall = recall_score(y_train_yes, y_train_pred)
    f1 = f1_score(y_train_yes, y_train_pred)
    accuracy = accuracy_score(y_train_yes, y_train_pred)
    precisions, recalls, thresholds = precision_recall_curve(y_train_yes, y_scores)

    print('Matrice de confusion : ', conf_matrix)
    print('Précision : ', precision)
    print('Recall : ', recall)
    print('F1-score : ', f1)
    print('Accuracy : ', accuracy) 

    plt.figure(figsize=(8, 4))
    plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
    plt.xlim([-700000, 700000])
    plt.show()
    



Mélange de données

In [ ]:
import numpy as np


shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train.iloc[shuffle_index], y_train.iloc[shuffle_index]

# Garder les données avec attrition à True (> 0)
y_train_yes = (y_train > 0)
y_test_yes = (y_test > 0 )

##### SDG Classifier

In [ ]:

from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, random_state=42)
sgd_clf.fit(X_train, y_train_yes)

predictions = sgd_clf.predict(X_test)
y_scores = sgd_clf.decision_function(X_test)

display_scores(y_test_yes, predictions, y_scores)



## Gridsearch
Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour un modèle de régression RandomForestRegressor.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [26], 'max_features': ['log2'], 'max_depth': [15]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
 
# 5 sous-jeux de cross-val, ça fait en tout (12+6)*5=90 tours d'entraînement 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train,y_train )

print(grid_search.best_params_)
print(grid_search.best_estimator_)

Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour le modèle SVR.

Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour le modèle de régression linéaire.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

param_grid = [
    {'fit_intercept': [True, False]}
]

lin_reg = LinearRegression()

grid_search = GridSearchCV(lin_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour le modèle de régression par arbre de décision (DecisionTreeRegressor).